#Caso de Uso: Fondo Voluntario de Pensiones
---
Jonathan Adrian Rosales Herrera

El fondo voluntario de pensiones es un producto que permite a sus clientes ahorro y optimización de su capital mediante inversión diversificada. Dado un caso hipotético de crisis en los mercados y aumento de la inflación, la compañía observa un aumento en los retiros de capital en dicho producto. En busca del bienestar sus clientes, quienes estarían descapitalizando al realizar retiros, la compañía requiere un modelo predictivo que muestre la probabilidad de que un CLIENTE retire el 70% o más de su saldo en los siguientes 3 meses

##Librerías

Importamos las librerías pertinentes

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix


##Datos

Cargamos cada archivo a utilizar

In [2]:
clientes = pd.read_parquet('/content/0clientes.parquet')
saldos = pd.read_parquet('/content/0saldos.parquet')
transferencias = pd.read_parquet('/content/0transferencias.parquet')

Observamos los primeros registros en cada conjunto de datos

In [3]:
clientes.head()

,TIPODOCUM,CIUDAD,FecNacim,EnvioExtractos,NroDocum
0,C,"BOGOTA D.C., BOGOTA",1954-12-16,E,1092070504
1,C,"BOGOTA D.C., BOGOTA",1969-08-04,E,1097627287
2,C,"BOGOTA D.C., BOGOTA",1961-08-15,T,1066619616
3,C,"BOGOTA D.C., BOGOTA",1986-02-08,T,1079454835
4,C,"MEDELLIN, ANTIOQUIA",1981-01-15,E,1199126816


In [4]:
saldos.head()

,TipoDocum,SALDO_202101,SALDO_2021O2,SALDO_2021O3,SALDO_2021O4,SALDO_2021O5,SALDO_2021O6,SALDO_2021O7,SALDO_2021O8,SALDO_2021O9,...,SALDO_202204,SALDO_202205,SALDO_202206,SALDO_202207,SALDO_202208,SALDO_202209,SALDO_202210,Contrato,PlanProducto,NroDocum
0,C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1921905,3376,1041211502
1,C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9441003,3376,1079794076
2,C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4580954,1722,1125051788
3,C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3549161,3376,1197460465
4,C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6619455,3376,1081082256


In [5]:
transferencias.head()

,NroEvento,NroTransaccion,FechaEfectiva,FechaProceso,ValorNeto,Usuario,CodDestino,CodTipoGiro,CodBancoDestinoACH,CodBancoSkandia,TipoCtaACH,TipoOper,EstadoReverso,Contrato,PlanProducto
0,210729985596,210729007960,2021-07-27,2021-07-29,-7921800.0,None,0,3,None,None,None,Directo,R,8774634,7256
1,210729986040,210729008404,2021-07-27,2021-07-29,-7850000.0,None,0,3,None,None,None,Directo,R,8774634,7256
2,210729985625,210729007989,2021-07-28,2021-07-29,-6000000.0,None,0,3,None,None,None,Directo,R,8774634,7256
3,210729985594,210729007958,2021-07-28,2021-07-29,-8000000.0,None,0,3,None,None,None,Directo,R,8774634,7256
4,210729986049,210729008413,2021-07-22,2021-07-29,-8256019.0,None,0,3,None,None,None,Directo,R,8774634,7256


In [6]:
data = pd.merge(transferencias, saldos, on=["Contrato", "PlanProducto"])
data.head()

,NroEvento,NroTransaccion,FechaEfectiva,FechaProceso,ValorNeto,Usuario,CodDestino,CodTipoGiro,CodBancoDestinoACH,CodBancoSkandia,...,SALDO_202202,SALDO_202203,SALDO_202204,SALDO_202205,SALDO_202206,SALDO_202207,SALDO_202208,SALDO_202209,SALDO_202210,NroDocum
0,210729985596,210729007960,2021-07-27,2021-07-29,-7921800.0,None,0,3,None,None,...,2.221431e+10,3.561328e+10,2.244195e+10,3.306558e+10,1.746658e+10,1.885348e+10,2.524539e+10,0.0,2.064662e+10,1179749878
1,210729986040,210729008404,2021-07-27,2021-07-29,-7850000.0,None,0,3,None,None,...,2.221431e+10,3.561328e+10,2.244195e+10,3.306558e+10,1.746658e+10,1.885348e+10,2.524539e+10,0.0,2.064662e+10,1179749878
2,210729985625,210729007989,2021-07-28,2021-07-29,-6000000.0,None,0,3,None,None,...,2.221431e+10,3.561328e+10,2.244195e+10,3.306558e+10,1.746658e+10,1.885348e+10,2.524539e+10,0.0,2.064662e+10,1179749878
3,210729985594,210729007958,2021-07-28,2021-07-29,-8000000.0,None,0,3,None,None,...,2.221431e+10,3.561328e+10,2.244195e+10,3.306558e+10,1.746658e+10,1.885348e+10,2.524539e+10,0.0,2.064662e+10,1179749878
4,210729986049,210729008413,2021-07-22,2021-07-29,-8256019.0,None,0,3,None,None,...,2.221431e+10,3.561328e+10,2.244195e+10,3.306558e+10,1.746658e+10,1.885348e+10,2.524539e+10,0.0,2.064662e+10,1179749878


In [27]:
data.isna().sum()

NroEvento         0
NroTransaccion    0
FechaEfectiva     0
FechaProceso      0
ValorNeto         0
                 ..
202206            0
202207            0
202208            0
202209            0
202210            0
Length: 61, dtype: int64

##Preprocesamiento

Calculamos el saldo neto mensual

In [7]:
for col in data.columns:
    if col.startswith("SALDO_"):
        mes = col.split("_")[1]
        data[mes] = data[col] + data["ValorNeto"]

Selección de las columnas relevantes para el análisis agrupadas por cliente y mes para sumar los valores netos y calcular el saldo neto mensual

In [11]:
saldo_mensual = data[["Contrato", "PlanProducto", "NroDocum", "FechaProceso", "ValorNeto"]].groupby(["NroDocum", pd.Grouper(key='FechaProceso', freq='M')])["ValorNeto"].sum().reset_index()

Cálculo del saldo acumulado de los últimos 3 meses

In [28]:
saldo_mensual["SaldoAcumulado"] = saldo_mensual.groupby("NroDocum")["ValorNeto"].rolling(window=3, min_periods=3).sum().reset_index(drop=True)

Detección de clientes que retiraron el 70% o más de su saldo en 3 meses


In [ ]:
saldo_mensual["Retiro70+"] = saldo_mensual["SaldoAcumulado"].shift(-3) <= -0.7 * saldo_mensual["SaldoAcumulado"]

Seleccionamos las características del modelo

In [29]:
features = saldo_mensual.dropna().set_index("NroDocum")[["SaldoAcumulado"]]

##Modelado

Establecemos la variable objetivo

In [30]:
target = saldo_mensual.dropna().set_index("NroDocum")["Retiro70+"]

Dividimos el conjunto en entrenamiento y prueba

In [22]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

Inicialización y ajuste del modelo de regresión logística

In [31]:
modelo = LogisticRegression()
modelo.fit(X_train, y_train)

LogisticRegression()

##Resultados

Obtenemos las predicciones del modelo

In [24]:
y_pred = modelo.predict(X_test)

Evaluamos los resultados

In [25]:
print("Matriz de Confusión:")
print(confusion_matrix(y_test, y_pred))
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred))

Matriz de Confusión:
[[15740  4524]
 [ 2037  9054]]

Reporte de Clasificación:
              precision    recall  f1-score   support

       False       0.89      0.78      0.83     20264
        True       0.67      0.82      0.73     11091

    accuracy                           0.79     31355
   macro avg       0.78      0.80      0.78     31355
weighted avg       0.81      0.79      0.79     31355



##Conclusion

 Se observa que en general el modelo tiene un buen rendimiento en la clasificación de clientes, con una precisión alta pero un recall ligeramente más bajo.

 Tanto para loas casos Falsos como positivos se tiene una tasa de predicción alta, teniendo un accuracy del 79%

 La matriz de confusión muestra una correcta presisión de casos positivos y negativos; sin embargo, presenta una cantidad considerable de falsos positivos, cuyo impacto sería importante considerar antes de realizar una toma de decisiones basada en este modelo